In [1]:
# Import libaries
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
%matplotlib inline

In [2]:
# Load the data
train = joblib.load('data//train.pkl')
validate = joblib.load('data//validate.pkl')
test = joblib.load('data//test.pkl')

In [3]:
train.columns = train.columns.astype(str)
validate.columns = validate.columns.astype(str)
test.columns = test.columns.astype(str)

In [4]:
# Split the data into X and y
X_train = train.drop('label', axis=1)
y_train = train.label

X_validate = validate.drop('label', axis=1)
y_validate = validate.label

X_test = test.drop('label', axis=1)
y_test = test.label

In [5]:
def evaluate(model, X, y):
    predictions = model.predict(X)
    score = accuracy_score(y, predictions)
    print(f'Model Accuracy: {round(score*100,3)} %\n')
    cm = confusion_matrix(y, predictions)
    print(f'Confusion Matrix:\n{cm}\n')
    report = classification_report(y, predictions,
            target_names=["Ham", "Spam"],output_dict=True,
            zero_division=np.nan)
    precision = report["Spam"]["precision"]
    recall = report["Spam"]["recall"]
    print(f'Precision (Spam): {round(precision*100,3)} %')
    print(f'Recall (Spam): {round(recall*100,3)} %')

In [6]:
def compare_models(models, X, y):
    model_scores = []
    for (key, value) in models.items():
        predictions = value.predict(X)
        clf_report = classification_report(y, predictions,
                    target_names=["Ham", "Spam"], output_dict=True,
                    zero_division=np.nan)
        precision = clf_report["Spam"]["precision"]
        recall =clf_report["Spam"]["recall"]
        score = accuracy_score(y, predictions)
        model_scores.append((key, round(score,3), round(precision,3),
                             round(recall,3), ))
    display(pd.DataFrame(model_scores,
    columns = ["Model", "Accuracy", "Precision", "Recall"]))

### Support Vector Classifier

In [7]:
svc = SVC(random_state=42).fit(X_train, y_train)

In [8]:
evaluate(svc, X_train, y_train)

Model Accuracy: 91.138 %

Confusion Matrix:
[[3488  121]
 [ 242  245]]

Precision (Spam): 66.94 %
Recall (Spam): 50.308 %


In [9]:
evaluate(svc, X_validate, y_validate)

Model Accuracy: 91.016 %

Confusion Matrix:
[[435  16]
 [ 30  31]]

Precision (Spam): 65.957 %
Recall (Spam): 50.82 %


### Gradient Boosted Trees

In [10]:
clf = GradientBoostingClassifier(random_state=42).fit(X_train, y_train)

In [11]:
evaluate(clf, X_train, y_train)

Model Accuracy: 98.804 %

Confusion Matrix:
[[3597   12]
 [  37  450]]

Precision (Spam): 97.403 %
Recall (Spam): 92.402 %


In [12]:
evaluate(clf, X_validate, y_validate)

Model Accuracy: 98.438 %

Confusion Matrix:
[[450   1]
 [  7  54]]

Precision (Spam): 98.182 %
Recall (Spam): 88.525 %


### Logistic Regression

In [13]:
reg = LogisticRegression(random_state=42).fit(X_train, y_train)

In [14]:
evaluate(reg, X_train, y_train)

Model Accuracy: 97.583 %

Confusion Matrix:
[[3593   16]
 [  83  404]]

Precision (Spam): 96.19 %
Recall (Spam): 82.957 %


In [15]:
evaluate(reg, X_validate, y_validate)

Model Accuracy: 98.242 %

Confusion Matrix:
[[450   1]
 [  8  53]]

Precision (Spam): 98.148 %
Recall (Spam): 86.885 %


In [16]:
model_list = {
    "Gradient Boosted Tree" : clf,
    "Multinomial Naive Bayes" : reg,
    "Support Vector Classifier": svc
}
compare_models(model_list, X_test, y_test)

,Model,Accuracy,Precision,Recall
0,Gradient Boosted Tree,0.977,0.962,0.836
1,Multinomial Naive Bayes,0.975,0.962,0.820
2,Support Vector Classifier,0.920,0.692,0.590


#### From the above data, the best model is Gradient Boosting Trees with an accuracy of 0.977 and precision of 0.962 and recall 0.836